In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

In [ ]:
# "Computs" center of "mass" for radial bins
def calc_rad(dat):
    # TODO Something is wrong here
    # starts at 0, didn't it start at 4?
    rad1,counts = np.unique(dat[:,0], return_counts=True)
    rad2 = np.unique(dat[:,1])
    rad = rad2-(rad2-rad1)/2.0
    counts = counts[0]
    return rad,counts
    

def Coord_Get(fl_in):
    '''
    Function to determine the radius and theta matrix
    plt_size is dependent on the size of the membrane plotted
    *** plt_size is no longer used remove ***
    
    dr, dth, theta, radius = Coord_Get(50)
    '''
    dat = np.loadtxt(fl_in, skiprows=1)
    Ntheta = np.shape(dat)[1]-4
    frames = 1
    #rad = dat[:,1]+(dat[:,1]-dat[:,0])/2.0    
    rad, frames = calc_rad(dat)
    dr = rad[1]-rad[0]
    the = np.linspace(0,2*np.pi,Ntheta)
    dth = the[1]-the[0]
    theta,radius=np.meshgrid(the,rad)

    return rad, dr, dth, theta, radius, frames, Ntheta

def get_header_info(fpath):
    '''
    Parameters
    ----------
    sat_files : str
        path to a data file. extracts extra information
        used for analysis

    Returns
    -------
    list
        Number of molecules, number of beads per mol, and expected
        density.
    '''
    line = open(fpath,'r').readline()
    num_mol = float(line.split(",")[0].split(":")[1].split(" ")[1])
    beads = float(line.split(",")[1].split(":")[1].split(" ")[1])
    avg_A = float(line.split(',')[2].split(":")[1].split(" ")[1])
    exrho = float(line.split(",")[3].split(":")[1].split("/")[0])
    avg_chain = float(line.split(",")[4].split(":")[1].split(" ")[1])
    return num_mol,avg_A,beads,exrho,avg_chain

In [ ]:
data_root = Path("../ELIC_PE_PG/trajectory_version/")
fpath = data_root.joinpath("POPG.dat.upp.dat")

In [ ]:
num_mol,avg_A,beads,exrho,avg_chain = get_header_info(fpath)
the_data = np.loadtxt(fpath, skiprows=1)
rad, dr, dth, theta, radius, frames, Ntheta = Coord_Get(fpath)


In [ ]:
def get_shell(some_data, inner_radius):
    mask = some_data[:,0] == inner_radius
    return some_data[mask, 3:-1]

In [ ]:
test_radius = 60
test_shell = get_shell(the_data, test_radius)


In [ ]:
frequencies = np.bincount(test_shell[:,:].astype(int).flatten())
densities = frequencies/np.sum(frequencies)
fig, ax = plt.subplots()
ax.plot(np.arange(len(frequencies)), densities)
ax.vlines(dr*dth*test_radius*exrho, 0, np.max(densities), color='k', linestyles='dashed')
ax.set_xlabel(r"$N_\mathrm{beads}$")
ax.set_ylabel("Probability Density")
ax.set_title(f"Probabiliy Density for a Bin at {test_radius} Angstroms")
plt.savefig("Sample_PDF.pdf")